In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [85]:
df_cleaned = main_cleaning(df_raw)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


In [86]:
df_cleaned = df_cleaned.dropna()         # Solo lo hago en el df_cleaned de ML

In [87]:
# df_cleaned.corr()

Results sin normnalizar -> (17.717622715597013, 12.581512831873468, 0.507506217561434)

In [88]:
features = df_cleaned[["Run", "px1", "py1", "pz1", "phi1", "eta1", "px2", "py2", "pz2","phi2", "eta2", "is_same_charge", "is_outlier"]]
target = df_cleaned[["M"]]

In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [90]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm= normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [91]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)   
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [92]:
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error, make_scorer


mse_scorer = make_scorer(root_mean_squared_error)
r2_scorer = make_scorer(r2_score)
mae_scorer = make_scorer(mean_absolute_error)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Define hyperparameter space for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200, 300, 400, 500],  # Número de árboles en el bosque
    'max_depth': [None, 10, 20, 30, 40, 50],        # Máxima profundidad de cada árbol
    'min_samples_split': [2, 5, 10],                # Número mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4],                  # Número mínimo de muestras requeridas para estar en una hoja
    'max_features': ['auto', 'sqrt', 'log2'],       # Número de características a considerar para la mejor división
    'bootstrap': [True, False],                     # Método para seleccionar muestras para entrenar cada árbol
    'criterion': ['absolute_error', 'friedman_mse', 'poisson', 'squared_error'],               # Función para medir la calidad de una división
}


# Initialize base XGBoost model
rf_regressor = RandomForestRegressor()

# Set up KFold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Configure GridSearchCV for RF
grid_search_rf = GridSearchCV(rf_regressor, param_grid_rf, cv=kfold, scoring='neg_mean_squared_error')

# Train GridSearchCV with XGBoost
grid_search_rf.fit(X_train, y_train)

# Extract the optimal XGBoost model
best_rf = grid_search_rf.best_estimator_

# Predict using the optimal model
y_pred_train_best_rf = best_rf.predict(X_train)
y_pred_test_best_rf = best_rf.predict(X_test)

# Cross-validation scores for XGBoost using best_xgb
mse_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=mse_scorer)
r2_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=r2_scorer)
mae_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=kfold, scoring=mae_scorer)

# Compile results into a DataFrame for XGBoost with Grid Search
evaluation_metrics_xgb = {
    'Model': ['RandomForest (Grid Search)'],
    'Avg_MSE_CV': [np.mean(mse_scores_rf)],
    'Std_MSE_CV': [np.std(mse_scores_rf)],
    'Avg_R2_CV': [np.mean(r2_scores_rf)],
    'Std_R2_CV': [np.std(r2_scores_rf)],
    'Avg_MAE_CV': [np.mean(mae_scores_rf)],
    'Std_MAE_CV': [np.std(mae_scores_rf)],
    'MSE_Train': [root_mean_squared_error(y_train, y_pred_train_best_rf)],
    'R2_Train': [r2_score(y_train, y_pred_train_best_rf)],
    'MAE_Train': [mean_absolute_error(y_train, y_pred_train_best_rf)],
    'MSE_Test': [root_mean_squared_error(y_test, y_pred_test_best_rf)],
    'R2_Test': [r2_score(y_test, y_pred_test_best_rf)],
    'MAE_Test': [mean_absolute_error(y_test, y_pred_test_best_rf)]
}

rf_results_df = pd.DataFrame(evaluation_metrics_xgb)

# # Append results to the existing DataFrame
# results_df = pd.concat([results_df, xgb_results_df], ignore_index=True)
# results_df

c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

In [1]:
rf_results_df

NameError: name 'rf_results_df' is not defined